In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
base_url = 'https://www.bbc.com/sport/football/european-championship/top-scorers'
urls = [base_url, base_url+'/assists']
dfs=[]
for url in urls:
    r = requests.get(url)
    soup = bs(r.text)
    header = soup.find('tr').find_all('th')
    players = soup.find('tbody').find_all('tr')
    
    def columns(tags):
        column = []
        for tag in tags: 
            if tag.find('span', class_='gs-u-vh'):
                column.append(tag.find('span', class_='gs-u-vh').text)
            else: column.append(tag.text)
        return column

    def body(tags):
        players_list = [] 
        for player in tags:
            ind_player = player.find_all('td')
            player_stats = [
                i.select_one('.sp-c-top-scorers__player-name').select_one('.gs-u-vh').text
                if i.select_one('.sp-c-top-scorers__player-name')
                else i.text for i in ind_player
            ]
            players_list.append(player_stats)
        return players_list
    
    dfs.append(pd.DataFrame(body(players), columns = columns(header)))

In [ ]:
df_g = dfs[0]   # Data Frame of Goal Scorers
df_g

In [ ]:
font_title = {'family': 'Comic Sans MS', 'color':'#030300', 'size': 22, 'weight': 'bold'}
font_legend = {'family': 'Comic Sans MS', 'color':'#030300', 'size': 14, 'weight': 'bold'}
font_axes = {'family': 'Comic Sans MS', 'color':'#131312', 'size': 14}

df_tg = df_g[df_g['Goals']>='3'] # Players with 3 or more goals
df_tg = df_tg.astype({'Name': str,'Played': int, 'Goals': int, 'Goals per 90': float})


def bar_chart(labels, values, title = None, theme = 'ggplot'):
    plt.style.use(theme)
    plt.figure(figsize=(12,6))
    plt.barh(labels, values, align ='center')
    plt.yticks(fontsize = 14, weight = 'bold')
    plt.gca().invert_yaxis()
    plt.title(title, fontdict = font_title)
    plt.grid()
    plt.show()

bar_chart(df_tg['Name'],df_tg['Goals'],title = 'Number of Goals by Players in UEFA Euro 2020')


In [ ]:
# Grouped Bar Chart 
plt.figure(figsize=(12,8))
df_tf = df_tg.iloc[:5,:] # top five players
labels = ['Games Played','Goals Scored', "Goals per 90'"]
players, played, goals, gp9 = df_tf['Name'], df_tf['Played'], df_tf['Goals'], df_tf['Goals per 90']
x = np.arange(len(played))
tick_pos = x
plt.style.use('ggplot')
w = 0.2
for i in [played, goals, gp9]:
    plt.bar(x,i, width = w)
    x = x + w
plt.legend(labels, loc = 0, fontsize = 14)
plt.title('Stats of top 5 Goal Scorers in UEFA Euro 2020', fontdict = font_title)
plt.xticks(tick_pos+(w+(w/4)),players,fontsize = 14, weight = 'bold')
plt.show()


## Assist Providers

In [ ]:
df_a = dfs[1] 
df_a.loc[3,'Name'] = 'P.E.Højbjerg'

In [ ]:
df_ta = df_a[df_a['Assists']>='1'] # PLayers with 2 or more assists
df_ta = df_ta.astype({'Name': str,'Played': int, 'Assists': int, 'Chances Created': int})
df_tc = df_ta.sort_values(by=['Chances Created'], ascending = False)

In [ ]:
bar_chart(df_ta['Name'],df_ta['Assists'],title = 'Number of Assists by Players in UEFA Euro 2020')

In [ ]:
plt.figure(figsize=(12,8))
df_ta = df_ta.iloc[:5,:] # top five players based on chances created
labels = ['Games Played','Assists Provided', 'Chances Created']
players, played, goals, gp9 = df_ta['Name'], df_ta['Played'], df_ta['Assists'], df_ta['Chances Created']
x = np.arange(len(played))
tick_pos = x
plt.style.use('seaborn')
w = 0.2
for i in [played, goals, gp9]:
    plt.bar(x,i, width = w)
    x = x + w
plt.legend(labels, loc = 0, fontsize = 14)
plt.title('Stats of top 5 Assists Providers in UEFA Euro 2020', fontdict = font_title)
plt.xticks(tick_pos+(w+(w/4)),players,fontsize = 14, weight = 'bold')
plt.show()

In [ ]:
bar_chart(df_tc['Name'],df_tc['Chances Created'],
          title = 'Number of Chances Created by Players in UEFA Euro 2020', theme = 'seaborn')